In [15]:
from typing import Tuple
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
import pandas as pd

import numpy as np

import torch

from torch import nn
from torch.utils.data import Dataset, DataLoader

from lightgbm import LGBMClassifier, LGBMModel

import wandb

In [3]:
# df = pd.concat([pd.read_csv("more-elements/more-elements.csv"), pd.read_csv("new-more-data/new-more-data.csv")])
# df
df = pd.read_pickle("half_data.pkl")
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,element_1,element_2,element_3,element_1_ratio,element_2_ratio,element_3_ratio,temp,pressure,air_ratio
0,1.402026e-21,1.650584e-21,1.910661e-21,2.264304e-21,2.829955e-21,3.673546e-21,4.745899e-21,5.773241e-21,6.446840e-21,6.741727e-21,6.582489e-21,5.906387e-21,4.936567e-21,3.969936e-21,3.157695e-21,2.580630e-21,2.223242e-21,2.026056e-21,1.946090e-21,1.955826e-21,2.040171e-21,2.206358e-21,2.478642e-21,2.844626e-21,3.309050e-21,4.034348e-21,5.249000e-21,7.346762e-21,1.102829e-20,1.260035e-20,1.144159e-20,1.102028e-20,1.019030e-20,8.599705e-21,6.897473e-21,5.743122e-21,5.167753e-21,4.805634e-21,4.353729e-21,3.815732e-21,...,2.582544e-20,2.721499e-20,3.124854e-20,3.789514e-20,4.669505e-20,5.807026e-20,6.981119e-20,7.275558e-20,6.391100e-20,5.402978e-20,4.967234e-20,4.965700e-20,5.311711e-20,6.169802e-20,7.290048e-20,7.594680e-20,6.555544e-20,5.148921e-20,3.991976e-20,3.151199e-20,2.640980e-20,2.431202e-20,2.492346e-20,2.830819e-20,3.440358e-20,4.219092e-20,5.094985e-20,5.958371e-20,6.033496e-20,4.892507e-20,3.466256e-20,17,23,-1,0.440157,0.559843,0.000000,283.0,0.775,0.0
1,4.102767e-22,4.371429e-22,4.698476e-22,5.105580e-22,5.626400e-22,6.315813e-22,7.266133e-22,8.610945e-22,1.048941e-21,1.323529e-21,1.790329e-21,2.741052e-21,5.153006e-21,1.408943e-20,4.529096e-20,3.450863e-20,1.097656e-20,4.470503e-21,2.616943e-21,1.883244e-21,1.552667e-21,1.414920e-21,1.397991e-21,1.483943e-21,1.688011e-21,2.056539e-21,2.671932e-21,3.734862e-21,6.107336e-21,1.420882e-20,4.434622e-20,6.934827e-20,2.113794e-20,7.670683e-21,4.151470e-21,2.810894e-21,2.196930e-21,1.903222e-21,1.787283e-21,1.797999e-21,...,3.946351e-20,4.777244e-20,3.931877e-20,3.235270e-20,3.415764e-20,4.589537e-20,7.292346e-20,1.059205e-19,1.030581e-19,6.795232e-20,3.991546e-20,2.559364e-20,1.825575e-20,1.392931e-20,1.100241e-20,9.010368e-21,7.601953e-21,6.558910e-21,5.773313e-21,5.166842e-21,4.685619e-21,4.296194e-21,3.974874e-21,3.697337e-21,3.449277e-21,3.232351e-21,3.043976e-21,2.878916e-21,2.732219e-21,2.600618e-21,2.481710e-21,0,19,-1,0.428047,0.571953,0.000000,303.0,0.800,0.6
2,3.610634e-23,2.739156e-23,1.628379e-23,1.195799e-23,1.069667e-23,1.058083e-23,1.121691e-23,1.274356e-23,1.515703e-23,1.882304e-23,2.765373e-23,5.280424e-23,9.994841e-23,1.025996e-22,5.069554e-23,2.723265e-23,1.974679e-23,1.598748e-23,1.433372e-23,1.372129e-23,1.367482e-23,1.410361e-23,1.510624e-23,1.691192e-23,1.875220e-23,2.240808e-23,3.220895e-23,5.393713e-23,6.510751e-23,5.954250e-23,8.271405e-23,9.200736e-23,9.122588e-23,1.331502e-22,8.646935e-23,6.185798e-23,6.638411e-23,8.779085e-23,1.617873e-22,3.211730e-22,...,5.019806e-21,4.129120e-21,4.602884e-21,3.254167e-21,2.033301e-21,1.581803e-21,1.610553e-21,2.027592e-21,3.167181e-21,6.387400e-21,1.310640e-20,1.753743e-20,2.199205e-20,1.023640e-20,4.576353e-21,3.017550e-21,1.812410e-21,1.168226e-21,8.550986e-22,6.841617e-22,5.926524e-22,5.327198e-22,4.750825e-22,4.604090e-22,4.928265e-22,5.918520e-22,8.413834e-22,1.550616e-21,3.519526e-21,5.865368e-21,2.756785e-21,2,11,13,0.342547,0.376666,0.280788,283.0,0.500,0.6
3,1.016487e-22,1.356304e-22,1.894228e-22,3.406116e-22,8.410447e-22,2.039525e-21,1.635235e-21,6.587985e-22,3.087047e-22,2.124933e-22,1.564710e-22,1.374398e-22,1.409981e-22,1.649231e-22,2.224252e-22,3.634082e-22,7.898927e-22,1.967533e-21,2.940770e-21,1.216788e-21,4.950690e-22,2.844850e-22,2.079753e-22,1.808446e-22,1.827857e-22,2.158678e-22,2.766854e-22,3.773140e-22,6.842961e-22,1.735018e-21,4.194633e-21,2.227487e-21,8.708983e-22,4.473654e-22,3.266422e-22,2.474372e-22,2.186360e-22,2.257415e-22,2.675181e-22,3.703709e-22,...,2.034911e-21,1.633748e-21,1.343361e-21,1.126574e-21,9.601437e-22,8.298215e-22,7.240965e-22,6.381964e-22,5.674321e-22,5.083574e-22,4.586411e-22,4.170241e-22,3.799903e-22,3.476131e-22,3.197439e

In [3]:
print(len(df))
df = df[(df[[str(i) for i in range(1001)]] > 0).all(axis=1)]
print(len(df))

183553
183553


In [4]:
len(df[(df["element_1"] == 0) | (df["element_2"] == 0) | (df["element_3"] == 0)]) / len(df)

0.11451733286843582

In [4]:
class CustomCatboostSpectraDataset(Dataset):
    def __init__(self, data: pd.DataFrame, device="cuda:0") -> None:
        self.data = data
        self.elements = self.data["element_1"].unique()
        self.air_ratios = data.air_ratio.to_numpy(dtype=np.float64)

        self.spectras = self.data[[str(i) for i in range(1001)]].to_numpy(
            dtype=np.float64
        )

        self.ratios = self.data[
            ["element_1_ratio", "element_2_ratio", "element_3_ratio"]
        ].to_numpy(dtype=np.float64)
        self.element_indices = self.data[
            ["element_1", "element_2", "element_3"]
        ].to_numpy(dtype=np.int32)

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx: int) -> Tuple[np.ndarray, np.ndarray]:
        spectra = np.log(self.spectras[idx])

        elements_distribution = np.zeros(
            [len(self.elements) + 1], dtype=np.float64
        )
        indices = self.element_indices[idx, :]
        indices = indices[indices != -1]
        elements_distribution[indices] = self.ratios[idx][range(indices.shape[0])] * (1 - self.air_ratios[idx])
        elements_distribution[-1] = self.air_ratios[idx]

        return spectra, elements_distribution

In [5]:
train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)

In [6]:
train_dataset = CustomCatboostSpectraDataset(train_df, device="cpu")
val_dataset = CustomCatboostSpectraDataset(val_df, device="cpu")

In [7]:
X_train, y_train = np.array([train_dataset[i][0] for i in range(len(train_dataset))]), np.array(
    [train_dataset[i][1] for i in range(len(train_dataset))]
)

X_val, y_val = np.array([val_dataset[i][0] for i in range(len(val_dataset))]), np.array(
    [val_dataset[i][1] for i in range(len(val_dataset))]
)

In [22]:
gb = MultiOutputRegressor(
    LGBMModel(
        device="gpu",
        n_estimators=1000,
        learning_rate=0.05,
        reg_lambda=10,
        subsample=0.9,
        objective="xentropy",
    )
)



gb.fit(X=X_train, y=y_train)

MultiOutputRegressor(estimator=LGBMModel(device='gpu', learning_rate=0.05,
                                         n_estimators=1000,
                                         objective='xentropy', reg_lambda=10,
                                         subsample=0.9))

In [24]:
def cross_entropy(predictions, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions.
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1.0 - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets * np.log(predictions + 1e-9)) / N
    return ce

In [25]:
from sklearn.metrics import (
    mean_absolute_error
)

In [26]:
# n_estimators=1000,
# learning_rate=0.05,
# reg_lambda=10,
# subsample=0.9,
# objective="xentropy",

pred = gb.predict(X_val)

print(f"MAE: {mean_absolute_error(y_val, pred)}")
print(f"cross_entropy: {cross_entropy(y_val, pred)}")

MAE: 0.00996983905543426
cross_entropy: 2.305713339714351


In [27]:
gb = MultiOutputRegressor(
    LGBMModel(
        device="gpu",
        objective="xentropy",
    )
)


gb.fit(X=X_train, y=y_train)
pred = gb.predict(X_val)

print(f"MAE: {mean_absolute_error(y_val, pred)}")
print(f"cross_entropy: {cross_entropy(y_val, pred)}")

MAE: 0.012276534385217875
cross_entropy: 2.647337031291116


In [28]:
gb = MultiOutputRegressor(
    LGBMModel(
        device="gpu",
        n_estimators=200,
        objective="xentropy",
    )
)


gb.fit(X=X_train, y=y_train)
pred = gb.predict(X_val)

print(f"MAE: {mean_absolute_error(y_val, pred)}")
print(f"cross_entropy: {cross_entropy(y_val, pred)}")

MAE: 0.011270461239920353
cross_entropy: 2.455917556660542


In [17]:
# device="gpu",
# n_estimators=200,
pred = gb.predict(X_val)
f1_score(y_val, pred, average="macro")

0.9140079621818451

In [19]:
# device="gpu",
# n_estimators=1000,
pred = gb.predict(X_val)
f1_score(y_val, pred, average="macro")

0.9198152912510928

In [21]:
# device="gpu",
# n_estimators=1000,
# learning_rate=0.5
pred = gb.predict(X_val)
f1_score(y_val, pred, average="macro")

0.7762751594106693

In [23]:
# device="gpu",
# n_estimators=1000,
# learning_rate=0.05
pred = gb.predict(X_val)
f1_score(y_val, pred, average="macro")

0.9198880481987812

In [25]:
# device="gpu",
# n_estimators=1000,
# learning_rate=0.05
# reg_lambda = 10
pred = gb.predict(X_val)
f1_score(y_val, pred, average="macro")

0.9200312033435948

In [27]:
# device="gpu",
# n_estimators=1000,
# learning_rate=0.05
# reg_lambda = 100
pred = gb.predict(X_val)
f1_score(y_val, pred, average="macro")

0.9174467214609332

In [9]:
# device="gpu",                             BEST!!!
# n_estimators=1000,
# learning_rate=0.05,
# reg_lambda=10,
# subsample=0.9,
pred = gb.predict(X_val)
print(f'F1: {f1_score(y_val, pred, average="macro")}')
print(f'precision: {precision_score(y_val, pred, average="macro")}')
print(f'recall: {recall_score(y_val, pred, average="macro")}')
print(f"accuracy: {accuracy_score(y_val, pred)}")

F1: 0.9200572722317097
precision: 0.9686400165182278
recall: 0.8942688357732655
accuracy: 0.688337631206189


In [10]:
print(f"F1: {f1_score(y_val, pred, average=None)}")
print(f"precision: {precision_score(y_val, pred, average=None)}")
print(f"recall: {recall_score(y_val, pred, average=None)}")

F1: [0.99790627 0.46636971 0.48387097 0.99701774 0.99601339 0.99825535
 0.88379646 0.96150113 0.35326714 0.97178683 0.98854284 0.9937946
 0.99616491 0.996139   0.99593298 0.99713193 0.99510964 0.99728997
 0.99855561 0.99706201 0.9900156  0.99191375 0.97874396 0.99397781
 0.9812722 ]
precision: [1.         0.79801829 0.77439887 1.         1.         0.99968234
 0.996744   0.99932751 0.65106383 1.         0.99967363 1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.99905542 0.99968041 0.99934232 1.
 0.99901381]
recall: [0.99582128 0.32945249 0.35186375 0.99405321 0.99205845 0.99683244
 0.79384117 0.92643392 0.24239544 0.94512195 0.9776572  0.98766574
 0.99235912 0.99230769 0.9918989  0.99428027 0.99026688 0.99459459
 0.99711538 0.99414123 0.98113791 0.98426683 0.9589776  0.98802773
 0.96414975]


In [35]:
pred, y_val[i]

(array([1.23883874e-06, 4.38560582e-03, 5.62113566e-03, 4.34981757e-04,
        1.37915759e-05, 1.60435102e-05, 6.15459019e-03, 5.16426915e-03,
        6.25508849e-03, 3.83966453e-03, 3.68702319e-04, 1.52557320e-04,
        1.44995356e-05, 1.01582786e-04, 1.56701221e-01, 3.78149110e-06,
        2.47053659e-01, 3.95344475e-04, 2.64859709e-04, 2.57998431e-05,
        2.71519559e-03, 2.26385942e-04, 3.70902956e-03, 1.02934236e-04,
        2.68017492e-02, 5.55836556e-01]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.19726331,
        0.        , 0.39952827, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.10320842,
        0.3       ]))

In [36]:
np.sum(np.abs(pred[y_val[i] != 0] - y_val[i][y_val[i] != 0])) / y_val[i][y_val[i] != 0].shape[0]

0.1313199816354742

In [14]:
pred_total = gb.predict_proba(X_val)

In [15]:
cross_entropy(pred_total, y_val)

1.237707421082145